### Libraries

In [1]:
import sys
sys.path.append(".")

In [2]:
from analyze import TestEvaluator
from modules.checklist.checklist import *

import pandas as pd
import pypandoc

## Checklist

In [3]:
# checklist_path = '../../checklist/checklist_demo.csv'
# checklist = Checklist(checklist_path, checklist_format=ChecklistFormat.CSV)
# checklist.as_markdown()

In [4]:
# pd.DataFrame({'a':[1, 2], 'b':[3, 4]})

## Inputs

Please specify the `test_functions_directory` below to load the ML test code base, the parameters, e.g. checklist, and the corresponding models to for evaluation

In [5]:
models = []

In [6]:
test_functions_directory = '../../../lightfm/tests'

In [7]:
# temperatures = [0.1]
# models = ['gpt-3.5-turbo']

In [8]:
checklist_directory = '../../checklist/checklist_demo.csv'

In [9]:
name = 'checklist_demo_1'
evaluator = TestEvaluator(test_functions_directory)
evaluator.load_checklist(checklist_directory)
models.append({'name': name, 'model': evaluator})

In [10]:
evaluator.evaluate()

100%|██████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:13<00:00,  6.95s/it]


In [25]:
result = dict()
result['score'] = evaluator.get_completeness_score(score_format='number')
result['report'] = evaluator.evaluation_report
result['result'] = evaluator.evaluation_result
result['temp'] = None

In [55]:
# TBC_FIXME From checklist.py. To be refactored 
def _get_md_representation(content: dict, curr_level: int):
    repeated_col = [k for k, v in content.items() if isinstance(v, list)]

    # print out header for each item
    md_repr = '#' * curr_level
    if 'ID' in content.keys():
        md_repr += f" {content['ID']}"
    if 'Title' in content.keys():
        md_repr += f" {content['Title']}\n\n"
    elif 'Topic' in content.keys():
        md_repr += f" {content['Topic']}\n\n"

    # print out non-title, non-repeated items
    for k, v in content.items():
        if k not in repeated_col and k not in ['Title', 'Topic', 'ID']:
            md_repr += f'**{k}**: {v}\n\n'

    # handle repeated columns and references
    point_form_cols = ['References', 'Function References', 'Observations']
    for k in repeated_col:
        if k not in point_form_cols:
            for item in content[k]:
                md_repr += _get_md_representation(item, curr_level=curr_level + 1)
        else:
            md_repr += f'**{k}:**\n\n' + '\n'.join(f'  - {item}' for item in content[k]) + '\n\n'

    return md_repr

# TBC_FIXME From checklist.py. To be refactored 
@staticmethod
def __filedump_check(output_path: str, exist_ok: bool):
    if not exist_ok and os.path.exists(output_path):
        raise FileExistsError("Output file already exists. Use `exist_ok=True` to overwrite.")
    return True

# TBC_FIXME From checklist.py. To be refactored 
def export_html(content: str, output_path: str, exist_ok: bool = False):
    # self.__filedump_check(output_path, exist_ok)
    pypandoc.convert_text(content, 'html', format='md', outputfile=output_path, extra_args=['--from=gfm'])

# TBC_FIXME From checklist.py. To be refactored 
def export_pdf(content: str, output_path: str, exist_ok: bool = False):
    # self.__filedump_check(output_path, exist_ok)
    pypandoc.convert_text(content, 'pdf', format='md', outputfile=output_path,
                          extra_args=['--pdf-engine=tectonic'])

def export_evaluation_report(output_path, format='html', exist_ok: bool = False):
    # if self.evaluation_report is None:
    #     raise NotImplementedError(
    #         # TBC_FIXME
    #         "Evaluation report is not yet created. Please make sure the function `get_completeness_score` is run before calling this export function"
    #     )

    score = result['score']
    summary_df = result['report'][['ID', 'Title', 'is_Satisfied', 'n_files_tested']]
    details = result['report'][['ID', 'Title', 'Requirement', 'Observations', 'Function References']].to_dict(orient='records')

    export_content = dict()
    export_content['Title'] = 'Test Evaluation Report'
    export_content['Report Areas'] = []
    export_content['Report Areas'].append({'Title': 'Summary', 'Completeness Score': score, 'Completeness Score per Checklist Item': '\n\n' + summary_df.to_markdown(index=False)})
    export_content['Report Areas'].append({'Title': 'Details', 'Report Detail': details})
    if format=='html':
        export_html(_get_md_representation(export_content, curr_level=1), output_path, exist_ok)
    elif format=='pdf':
        export_pdf(_get_md_representation(export_content, curr_level=1), output_path, exist_ok)
    return

In [48]:
result['report']

,,Requirement,is_Satisfied,n_files_tested,Observations,Function References
ID,Title,,,,,
1.1,Write Descriptive Test Names,"Each test function should have a clear, descri...",1.0,6,[(test_fast_functions.py) The test function 't...,[{'File Path': '../../../lightfm/tests/test_fa...
1.2,Keep Tests Focused,"Each test should focus on a single scenario, u...",1.0,6,[(test_fast_functions.py) The test function 't...,[{'File Path': '../../../lightfm/tests/test_fa...
2.1,Ensure Data File Loads as Expected,Ensure that data-loading functions correctly l...,1.0,6,"[(test_fast_functions.py) Not Applicable, nan,...",[{'File Path': '../../../lightfm/tests/test_fa...
5.1,Validate Model Input and Output Compatibility,Confirm that the model accepts inputs of the c...,1.0,6,[(test_fast_functions.py) The test functions v...,[{'File Path': '../../../lightfm/tests/test_fa...


In [12]:
output_path = '../../report/evaluation_report.html'
evaluator.get_completeness_score()
evaluator.export_evaluation_report(output_path, 'html', exist_ok=True)
# export_evaluation_report(output_path, 'pdf', exist_ok=True)

In [23]:
# result['report'].reset_index().to_dict(orient='records')
summary_df = result['report'].reset_index()[['ID', 'Title', 'is_Satisfied', 'n_files_tested']]
details = result['report'].reset_index()[['ID', 'Title', 'Requirement', 'References']].to_dict(orient='records')

In [24]:
summary_df

,ID,Title,is_Satisfied,n_files_tested
0,1.1,Write Descriptive Test Names,1.0,2
1,1.2,Keep Tests Focused,1.0,2
2,2.1,Ensure Data File Loads as Expected,0.5,2
3,5.1,Validate Model Input and Output Compatibility,1.0,2


In [25]:
export_content = dict()
export_content['Title'] = 'Test Evaluation Report'
export_content['Report Areas'] = []
export_content['Report Areas'].append({'Title': 'Summary', 
                                            'Completeness Score': result['score'], 
                                            'Completeness Score per Checklist Item': '\n' + summary_df.to_markdown(index=False)
                                            })
export_content['Report Areas'].append({'Title': 'Details', 'Report Detail': details})
export_content

{'Title': 'Test Evaluation Report',
 'Report Areas': [{'Title': 'Summary',
   'Completeness Score': 0.875,
   'Completeness Score per Checklist Item': '\n|   ID | Title                                         |   is_Satisfied |   n_files_tested |\n|-----:|:----------------------------------------------|---------------:|-----------------:|\n|  1.1 | Write Descriptive Test Names                  |            1   |                2 |\n|  1.2 | Keep Tests Focused                            |            1   |                2 |\n|  2.1 | Ensure Data File Loads as Expected            |            0.5 |                2 |\n|  5.1 | Validate Model Input and Output Compatibility |            1   |                2 |'},
  {'Title': 'Details',
   'Report Detail': [{'ID': '1.1',
     'Title': 'Write Descriptive Test Names',
     'Requirement': "Each test function should have a clear, descriptive name that accurately reflects the test's purpose and the specific functionality or scenario it examines

In [15]:
result['result']

[{'file': '../../../lightfm/tests/test_fast_functions.py',
  'report': [{'ID': '1.1',
    'Title': 'Write Descriptive Test Names',
    'Requirement': "Each test function should have a clear, descriptive name that accurately reflects the test's purpose and the specific functionality or scenario it examines.",
    'Observation': "The test function 'test_in_positives' has a descriptive name that indicates it tests for positive cases related to the input data.",
    'Functions': ['test_in_positives'],
    'Evaluation': 'Satisfied',
    'Score': 1,
    'file': '../../../lightfm/tests/test_fast_functions.py'},
   {'ID': '1.2',
    'Title': 'Keep Tests Focused',
    'Requirement': 'Each test should focus on a single scenario, using only one set of mock data and testing one specific behavior or outcome to ensure clarity and isolate issues.',
    'Observation': "The 'test_in_positives' test function focuses on testing the behavior related to positive cases in the input data.",
    'Functions': 

In [58]:
def get_completeness_score(score_format='fraction', verbose=False):
    # report_df = pd.DataFrame(self.evaluation_result)['report'].explode('report').apply(pd.Series)
    report_df = pd.DataFrame(result['result'])['report'].explode('report').apply(pd.Series)
    report_df = report_df.rename(columns={"file": "File Path"})
    report_df['Function References'] = report_df[['File Path', 'Functions']].to_dict(orient='records')
    report_df['Observation'] = '(' + report_df['File Path'].apply(lambda x: os.path.split(x)[-1]) + ') ' + report_df['Observation']
    report_df = report_df.groupby(['ID', 'Title']).agg({
        'Requirement': ['max'],
        'Score': ['max', 'count'],
        'Observation': [list],
        'Function References': [list],
    })
    report_df.columns = ['Requirement', 'is_Satisfied', 'n_files_tested', 'Observations', 'Function References']
    # self.evaluation_report = report_df
    result['report'] = report_df.reset_index()

    if score_format == 'fraction':
        score = f"{report_df['is_Satisfied'].sum()}/{report_df['is_Satisfied'].count()}"
    elif score_format == 'number':
        score = report_df['is_Satisfied'].sum()/report_df['is_Satisfied'].count()

    if verbose:
        print("Report:")
        print(report_df)
        print()
        print(f'Score: {score}')
        print()
    return score

In [61]:
result['score'] = get_completeness_score()
import json
# result['report']['function_sources'] = result['report']['function_sources'].apply(lambda x: '\n'.join([json.dumps(i) for i in x]))
print(result['report'])

    ID                                          Title  \
0  1.1                   Write Descriptive Test Names   
1  1.2                             Keep Tests Focused   
2  2.1             Ensure Data File Loads as Expected   
3  5.1  Validate Model Input and Output Compatibility   

                                         Requirement  is_Satisfied  \
0  Each test function should have a clear, descri...           1.0   
1  Each test should focus on a single scenario, u...           1.0   
2  Ensure that data-loading functions correctly l...           1.0   
3  Confirm that the model accepts inputs of the c...           1.0   

   n_files_tested                                       Observations  \
0               6  [(test_fast_functions.py) The test function 't...   
1               6  [(test_fast_functions.py) The test function 't...   
2               6  [(test_fast_functions.py) Not Applicable, nan,...   
3               6  [(test_fast_functions.py) The test functions v...   

  

In [50]:
def export_evaluation_report_to_html(output_path):
    if result['report'] is None:
        raise NotImplementedError(
            "Evaluation report is not yet created. Please make sure the function `get_completeness_score` is run before calling this export function"
        )
    else:
        report_df = result['report']
        # report_df['function_sources'] = report_df['function_sources'].apply(lambda x: '\n'.join([json.dumps(i) for i in x]))
        report_df['function_sources'] = pd.DataFrame(report_df['function_sources'])
        return report_df
        # with open(output_path, 'w') as f:
        #     f.write(report_df.to_html())

In [44]:
output_path = '../../report/evaluation_report.html'

In [51]:
export_evaluation_report_to_html(output_path)

,,is_Satisfied,n_files_tested,function_sources
ID,Title,,,
1.1,Write Descriptive Test Names,1.0,6,[{'File_Path': '../../../lightfm/tests/test_fa...
1.2,Keep Tests Focused,1.0,6,[{'File_Path': '../../../lightfm/tests/test_fa...
2.1,Ensure Data File Loads as Expected,1.0,6,[{'File_Path': '../../../lightfm/tests/test_fa...
5.1,Validate Model Input and Output Compatibility,1.0,6,[{'File_Path': '../../../lightfm/tests/test_fa...


In [16]:
with open(output_path, 'w') as f:
    f.write(result['report'].to_html())

Incorporate the data, prompts and parameters, feed into OpenAI API for test runs and fetch responses.

The evaluation will be based on 2 metrics calculated from the response:
- Completeness Score distribution: The distribution of the `num_test_runs` completeness scores per each set of parameters
- Consistency Score: Out of all `checklist` items, the proportion of results remain consistent among `num_test_runs` runs per each set of parameters

In [67]:
consistency_evaluator.get_completeness_score_dist()

test_no,1,2,3,4,5
model_name,,,,,
checklist_demo_1,1.0,1.0,1.0,1.0,1.0
checklist_demo_2,1.0,1.0,1.0,1.0,1.0


In [51]:
# import matplotlib
# completeness_score_df.plot(kind='box')

In [68]:
consistency_evaluator.get_consistency_dist()

test_no                 1    2    3    4    5  consistency
model_name       ID                                       
checklist_demo_1 1.1  1.0  1.0  1.0  1.0  1.0         True
                 1.2  1.0  1.0  1.0  1.0  1.0         True
                 2.1  1.0  1.0  1.0  1.0  1.0         True
                 5.1  1.0  1.0  1.0  1.0  1.0         True
checklist_demo_2 1.1  1.0  1.0  1.0  1.0  1.0         True
                 1.2  1.0  1.0  1.0  1.0  1.0         True
                 2.1  1.0  1.0  1.0  1.0  1.0         True
                 5.1  1.0  1.0  1.0  1.0  1.0         True

In [20]:
# consistency_df.groupby(['model_name']).agg({'consistency': 'mean'})